# Table of Contents
 <p><div class="lev2 toc-item"><a href="#borrowed-features" data-toc-modified-id="borrowed-features-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>borrowed features</a></div><div class="lev2 toc-item"><a href="#my-features-013" data-toc-modified-id="my-features-013-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>my features 013</a></div><div class="lev3 toc-item"><a href="#buro_bal_wide_013" data-toc-modified-id="buro_bal_wide_013-021"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>buro_bal_wide_013</a></div><div class="lev3 toc-item"><a href="#buro_wide_013" data-toc-modified-id="buro_wide_013-022"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>buro_wide_013</a></div><div class="lev3 toc-item"><a href="#buro_full_wide_013" data-toc-modified-id="buro_full_wide_013-023"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>buro_full_wide_013</a></div><div class="lev3 toc-item"><a href="#pos_bal_wide_013" data-toc-modified-id="pos_bal_wide_013-024"><span class="toc-item-num">0.2.4&nbsp;&nbsp;</span>pos_bal_wide_013</a></div><div class="lev3 toc-item"><a href="#cc_bal_wide_013" data-toc-modified-id="cc_bal_wide_013-025"><span class="toc-item-num">0.2.5&nbsp;&nbsp;</span>cc_bal_wide_013</a></div><div class="lev3 toc-item"><a href="#buro_bal_timestep1_013" data-toc-modified-id="buro_bal_timestep1_013-026"><span class="toc-item-num">0.2.6&nbsp;&nbsp;</span>buro_bal_timestep1_013</a></div><div class="lev3 toc-item"><a href="#buro_bal_timestep2_013" data-toc-modified-id="buro_bal_timestep2_013-027"><span class="toc-item-num">0.2.7&nbsp;&nbsp;</span>buro_bal_timestep2_013</a></div><div class="lev3 toc-item"><a href="#pos_bal_timestep1_013" data-toc-modified-id="pos_bal_timestep1_013-028"><span class="toc-item-num">0.2.8&nbsp;&nbsp;</span>pos_bal_timestep1_013</a></div><div class="lev3 toc-item"><a href="#pos_bal_timestep2_013" data-toc-modified-id="pos_bal_timestep2_013-029"><span class="toc-item-num">0.2.9&nbsp;&nbsp;</span>pos_bal_timestep2_013</a></div><div class="lev3 toc-item"><a href="#cc_bal_timestep1_013" data-toc-modified-id="cc_bal_timestep1_013-0210"><span class="toc-item-num">0.2.10&nbsp;&nbsp;</span>cc_bal_timestep1_013</a></div><div class="lev3 toc-item"><a href="#cc_bal_timestep2_013" data-toc-modified-id="cc_bal_timestep2_013-0211"><span class="toc-item-num">0.2.11&nbsp;&nbsp;</span>cc_bal_timestep2_013</a></div><div class="lev2 toc-item"><a href="#nmf-features-013" data-toc-modified-id="nmf-features-013-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>nmf features 013</a></div><div class="lev2 toc-item"><a href="#single-xgb" data-toc-modified-id="single-xgb-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>single xgb</a></div><div class="lev3 toc-item"><a href="#train-test-split" data-toc-modified-id="train-test-split-041"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>train test split</a></div><div class="lev3 toc-item"><a href="#nmf-features" data-toc-modified-id="nmf-features-042"><span class="toc-item-num">0.4.2&nbsp;&nbsp;</span>nmf features</a></div><div class="lev3 toc-item"><a href="#nmf-+-first-4" data-toc-modified-id="nmf-+-first-4-043"><span class="toc-item-num">0.4.3&nbsp;&nbsp;</span>nmf + first 4</a></div><div class="lev3 toc-item"><a href="#append-features" data-toc-modified-id="append-features-044"><span class="toc-item-num">0.4.4&nbsp;&nbsp;</span>append features</a></div><div class="lev2 toc-item"><a href="#stacking" data-toc-modified-id="stacking-05"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>stacking</a></div>

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from sklearn.model_selection import KFold
# from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import gc
import os

In [2]:
def save_dataframe(path, dataframe):
    np.save(path + ".data", dataframe.values)
    np.save(path + ".header", dataframe.columns)
    
def load_dataframe(path):
    data = np.load(path + ".data.npy")
    header = np.load(path + ".header.npy")
    return pd.DataFrame(data=data, columns=header)

def save_dataframe32(path, dataframe, keep=[]):
    col64 = [col_ for col_ in dataframe.columns if col_ in keep]
    col32 = [col_ for col_ in dataframe.columns if col_ not in keep]
    dataframe64 = dataframe[col64]
    dataframe32 = dataframe[col32]
    np.save(path + ".data64", dataframe64.values)
    np.save(path + ".header64", col64)
    np.save(path + ".data32", dataframe32.values.astype(np.float32))
    np.save(path + ".header32", col32)

def load_dataframe32(path):
    path_data32 = path + ".data32.npy"
    path_header32 = path + ".header32.npy"
    path_data64 = path + ".data64.npy"
    path_header64 = path + ".header64.npy"
    result = pd.DataFrame()
    if os.path.exists(path_data32):
        data32 = np.load(path_data32)
        header32 = np.load(path_header32)
        df32 = pd.DataFrame(data=data32, columns=header32)
        result = pd.concat([result, df32], axis=1)
    if os.path.exists(path_data64):
        data64 = np.load(path_data64)
        header64 = np.load(path_header64)
        df64 = pd.DataFrame(data=data64, columns=header64)
        result = pd.concat([result, df64], axis=1)
    return result

## borrowed features

1. https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code

2. https://www.kaggle.com/ogrellier/fork-lightgbm-with-simple-features/code

In [3]:
[f_ for f_ in os.listdir("./data/") if f_.endswith(".csv")]

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'buro_bal_wide_004.csv',
 'buro_full_wide_002.csv',
 'buro_full_wide_004.csv',
 'buro_wide_002.csv',
 'cc_bal_wide_004.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'pos_bal_wide_004.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [4]:
buro_bal = pd.read_csv('./data/bureau_balance.csv')
print('Buro bal shape : ', buro_bal.shape)

print('transform to dummies')
buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')], axis=1).drop('STATUS', axis=1)

print('Counting buros')
buro_counts = buro_bal[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
buro_bal['buro_count'] = buro_bal['SK_ID_BUREAU'].map(buro_counts['MONTHS_BALANCE'])

print('averaging buro bal')
avg_buro_bal = buro_bal.groupby('SK_ID_BUREAU').mean()

avg_buro_bal.columns = ['avg_buro_' + f_ for f_ in avg_buro_bal.columns]
del buro_bal
gc.collect()

print('Read Bureau')
buro = pd.read_csv('./data/bureau.csv')

print('Go to dummies')
buro_credit_active_dum = pd.get_dummies(buro.CREDIT_ACTIVE, prefix='ca_')
buro_credit_currency_dum = pd.get_dummies(buro.CREDIT_CURRENCY, prefix='cu_')
buro_credit_type_dum = pd.get_dummies(buro.CREDIT_TYPE, prefix='ty_')

buro_full = pd.concat([buro, buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum], axis=1)

del buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum
gc.collect()

print('Merge with buro avg')
buro_full = buro_full.merge(right=avg_buro_bal.reset_index(), how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))
# buro_full.columns = ['buro_' + f_ for f_ in buro_full.columns]

print('Counting buro per SK_ID_CURR')
nb_bureau_per_curr = buro_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
buro_full['SK_ID_BUREAU'] = buro_full['SK_ID_CURR'].map(nb_bureau_per_curr['SK_ID_BUREAU'])

print('Averaging bureau')
avg_buro = buro_full.groupby('SK_ID_CURR').mean()
# print(avg_buro.head())

del buro, buro_full
gc.collect()

print('Read prev')
prev = pd.read_csv('./data/previous_application.csv')

prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]

print('Go to dummies')
prev_dum = pd.DataFrame()
for f_ in prev_cat_features:
    prev_dum = pd.concat([prev_dum, pd.get_dummies(prev[f_], prefix=f_).astype(np.uint8)], axis=1)

prev = pd.concat([prev, prev_dum], axis=1)

del prev_dum
gc.collect()

print('Counting number of Prevs')
nb_prev_per_curr = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev['SK_ID_PREV'] = prev['SK_ID_CURR'].map(nb_prev_per_curr['SK_ID_PREV'])

print('Averaging prev')
avg_prev = prev.groupby('SK_ID_CURR').mean()
print(avg_prev.head())
del prev
gc.collect()

print('Reading POS_CASH')
pos = pd.read_csv('./data/POS_CASH_balance.csv')

print('Go to dummies')
pos = pd.concat([pos, pd.get_dummies(pos['NAME_CONTRACT_STATUS'])], axis=1)

print('Compute nb of prevs per curr')
nb_prevs = pos[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
pos['SK_ID_PREV'] = pos['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

print('Go to averages')
avg_pos = pos.groupby('SK_ID_CURR').mean()

del pos, nb_prevs
gc.collect()

print('Reading CC balance')
cc_bal = pd.read_csv('./data/credit_card_balance.csv')

print('Go to dummies')
cc_bal = pd.concat([cc_bal, pd.get_dummies(cc_bal['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')], axis=1)

nb_prevs = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
cc_bal['SK_ID_PREV'] = cc_bal['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

print('Compute average')
avg_cc_bal = cc_bal.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]

del cc_bal, nb_prevs
gc.collect()

print('Reading Installments')
inst = pd.read_csv('./data/installments_payments.csv')
nb_prevs = inst[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
inst['SK_ID_PREV'] = inst['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

avg_inst = inst.groupby('SK_ID_CURR').mean()
avg_inst.columns = ['inst_' + f_ for f_ in avg_inst.columns]

print('Read data and test')
data = pd.read_csv('./data/application_train.csv')
test = pd.read_csv('./data/application_test.csv')

inc_by_org = data.append(test)[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
docs = [_f for _f in data.columns if 'FLAG_DOC' in _f]
live = [_f for _f in data.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

print('Shapes : ', data.shape, test.shape)

print("New Features")
data = data[data['CODE_GENDER'] != 'XNA']

data['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
data['NEW_CREDIT_TO_ANNUITY_RATIO'] = data['AMT_CREDIT'] / data['AMT_ANNUITY']
data['NEW_CREDIT_TO_GOODS_RATIO'] = data['AMT_CREDIT'] / data['AMT_GOODS_PRICE']
data['NEW_DOC_IND_KURT'] = data[docs].kurtosis(axis=1)
data['NEW_LIVE_IND_SUM'] = data[live].sum(axis=1)
data['NEW_INC_PER_CHLD'] = data['AMT_INCOME_TOTAL'] / (1 + data['CNT_CHILDREN'])
data['NEW_INC_BY_ORG'] = data['ORGANIZATION_TYPE'].map(inc_by_org)
data['NEW_EMPLOY_TO_BIRTH_RATIO'] = data['DAYS_EMPLOYED'] / data['DAYS_BIRTH']
data['NEW_ANNUITY_TO_INCOME_RATIO'] = data['AMT_ANNUITY'] / (1 + data['AMT_INCOME_TOTAL'])
data['NEW_SOURCES_PROD'] = data['EXT_SOURCE_1'] * data['EXT_SOURCE_2'] * data['EXT_SOURCE_3']
data['NEW_EXT_SOURCES_MEAN'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
data['NEW_SCORES_STD'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
data['NEW_SCORES_STD'] = data['NEW_SCORES_STD'].fillna(data['NEW_SCORES_STD'].mean())
data['NEW_CAR_TO_BIRTH_RATIO'] = data['OWN_CAR_AGE'] / data['DAYS_BIRTH']
data['NEW_CAR_TO_EMPLOY_RATIO'] = data['OWN_CAR_AGE'] / data['DAYS_EMPLOYED']
data['NEW_PHONE_TO_BIRTH_RATIO'] = data['DAYS_LAST_PHONE_CHANGE'] / data['DAYS_BIRTH']
data['NEW_PHONE_TO_EMPLOY_RATIO'] = data['DAYS_LAST_PHONE_CHANGE'] / data['DAYS_EMPLOYED']
data['NEW_CREDIT_TO_INCOME_RATIO'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']

test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
test['NEW_CREDIT_TO_ANNUITY_RATIO'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']
test['NEW_CREDIT_TO_GOODS_RATIO'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']
test['NEW_DOC_IND_KURT'] = test[docs].kurtosis(axis=1)
test['NEW_LIVE_IND_SUM'] = test[live].sum(axis=1)
test['NEW_INC_PER_CHLD'] = test['AMT_INCOME_TOTAL'] / (1 + test['CNT_CHILDREN'])
test['NEW_INC_BY_ORG'] = test['ORGANIZATION_TYPE'].map(inc_by_org)
test['NEW_EMPLOY_TO_BIRTH_RATIO'] = test['DAYS_EMPLOYED'] / test['DAYS_BIRTH']
test['NEW_ANNUITY_TO_INCOME_RATIO'] = test['AMT_ANNUITY'] / (1 + test['AMT_INCOME_TOTAL'])
test['NEW_SOURCES_PROD'] = test['EXT_SOURCE_1'] * test['EXT_SOURCE_2'] * test['EXT_SOURCE_3']
test['NEW_EXT_SOURCES_MEAN'] = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
test['NEW_SCORES_STD'] = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
test['NEW_SCORES_STD'] = test['NEW_SCORES_STD'].fillna(test['NEW_SCORES_STD'].mean())
test['NEW_CAR_TO_BIRTH_RATIO'] = test['OWN_CAR_AGE'] / test['DAYS_BIRTH']
test['NEW_CAR_TO_EMPLOY_RATIO'] = test['OWN_CAR_AGE'] / test['DAYS_EMPLOYED']
test['NEW_PHONE_TO_BIRTH_RATIO'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_BIRTH']
test['NEW_PHONE_TO_EMPLOY_RATIO'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_EMPLOYED']
test['NEW_CREDIT_TO_INCOME_RATIO'] = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']

y = data[["SK_ID_CURR", 'TARGET']]
del data['TARGET']

categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]
categorical_feats
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

data = data.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')

del avg_buro, avg_prev
gc.collect()

Buro bal shape :  (27299925, 3)
transform to dummies
Counting buros
averaging buro bal
Read Bureau
Go to dummies
Merge with buro avg
Counting buro per SK_ID_CURR
Averaging bureau
Read prev
Go to dummies
Counting number of Prevs
Averaging prev
            SK_ID_PREV  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  \
SK_ID_CURR                                                         
100001             1.0     3951.000         24835.50    23787.00   
100002             1.0     9251.775        179055.00   179055.00   
100003             3.0    56553.990        435436.50   484191.00   
100004             1.0     5357.250         24282.00    20106.00   
100005             2.0     4813.200         22308.75    20076.75   

            AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  HOUR_APPR_PROCESS_START  \
SK_ID_CURR                                                               
100001                2520.0          24835.5                13.000000   
100002                   0.0         179055.0             

261

In [5]:
y.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


In [6]:
import sys

In [7]:
print(sys.getsizeof(data))
print(sys.getsizeof(test))
print(sys.getsizeof(y))

976642256
154810968
7380192


In [8]:
save_dataframe32("./bindata/data_013", data)
save_dataframe32("./bindata/test_013", test)
save_dataframe32("./bindata/y_013", y)

## my features 013

### buro_bal_wide_013

In [9]:
buro_bal = pd.read_csv('./data/bureau_balance.csv')

data_stack = {}

# total group
grp = buro_bal.groupby("SK_ID_BUREAU")
data_stack["cnt"] = grp.size()
data_stack["status_cnt"] = grp.STATUS.count()
data_stack["max_month"] = grp.MONTHS_BALANCE.max()
data_stack["min_month"] = grp.MONTHS_BALANCE.min()
data_stack["maxmin_month"] = data_stack["max_month"] - data_stack["min_month"]

# status group
for status_ in buro_bal.STATUS.unique():
    grp = buro_bal[buro_bal.STATUS == status_].groupby("SK_ID_BUREAU")
    data_stack["status_{}_cnt".format(status_)] = grp.size()
    data_stack["status_{}_max_month".format(status_)] = grp.MONTHS_BALANCE.max()
    data_stack["status_{}_min_month".format(status_)] = grp.MONTHS_BALANCE.min()
    data_stack["status_{}_maxmin_month".format(status_)] = data_stack["status_{}_max_month".format(status_)] - data_stack["status_{}_min_month".format(status_)]
    data_stack["status_{}_max_offset".format(status_)] = data_stack["max_month"] - data_stack["status_{}_max_month".format(status_)]
    data_stack["status_{}_min_offset".format(status_)] = data_stack["min_month"] - data_stack["status_{}_maxmin_month".format(status_)]
    data_stack["status_{}_raito".format(status_)] = data_stack["status_{}_cnt".format(status_)] / data_stack["cnt"]

buro_bal_wide = pd.DataFrame(data_stack)
print(buro_bal_wide.shape)
save_dataframe32("./bindata/buro_bal_wide_013", buro_bal_wide.reset_index())

del buro_bal, grp, data_stack
gc.collect()

(817395, 61)


104

### buro_wide_013

In [10]:
buro = pd.read_csv('./data/bureau.csv')
data_stack = {}

# total group
grp = buro.groupby("SK_ID_CURR")
data_stack["buro_cnt"] = grp.size()
data_stack["buro_type_cnt"] = grp.CREDIT_TYPE.count()
data_stack["buro_currency_cnt"] = grp.CREDIT_CURRENCY.count()

# active group
for status_ in buro.CREDIT_ACTIVE.unique():
    grp = buro[buro.CREDIT_ACTIVE == status_].groupby("SK_ID_CURR")
    data_stack["buro_active_{}_cnt".format(status_)] = grp.size()
    data_stack["buro_active_{}_ratio".format(status_)] = data_stack["buro_active_{}_cnt".format(status_)] / data_stack["buro_cnt"]
    data_stack["buro_active_{}_type_cnt".format(status_)] = grp.CREDIT_TYPE.count()
    data_stack["buro_active_{}_currency_cnt".format(status_)] = grp.CREDIT_CURRENCY.count()

# type group
for status_ in buro.CREDIT_TYPE.unique():
    grp = buro[buro.CREDIT_TYPE == status_].groupby("SK_ID_CURR")
    data_stack["buro_type_{}_cnt".format(status_)] = grp.size()
    data_stack["buro_type_{}_ratio".format(status_)] = data_stack["buro_type_{}_cnt".format(status_)] / data_stack["buro_cnt"]
    data_stack["buro_type_{}_type_cnt".format(status_)] = grp.CREDIT_TYPE.count()
    data_stack["buro_type_{}_currency_cnt".format(status_)] = grp.CREDIT_CURRENCY.count()

buro_wide = pd.DataFrame(data_stack)
print(buro_wide.shape)
save_dataframe32("./bindata/buro_wide_013", buro_wide.reset_index())

del buro_wide, data_stack, grp
gc.collect()

(305811, 79)


201

### buro_full_wide_013

In [15]:
buro = pd.read_csv('./data/bureau.csv')
buro_bal_wide = load_dataframe32("./bindata/buro_bal_wide_013")
buro_full = buro.merge(right=buro_bal_wide.reset_index(), how="left", on="SK_ID_BUREAU")

buro_full_numcols = list(buro_full.dtypes[buro_full.dtypes != "object"].index)
buro_full_numcols.remove("SK_ID_CURR")
if "SK_ID_BUREAU" in buro_full_numcols:
    buro_full_numcols.remove("SK_ID_BUREAU")

grp = buro_full.groupby("SK_ID_CURR")[buro_full_numcols]
buro_full_avg = grp.mean()
buro_full_max = grp.max()
buro_full_min = grp.min()
buro_full_maxmin = buro_full_max - buro_full_min

buro_full_avg.columns = [col_ + "_avg" for col_ in buro_full_avg.columns]
buro_full_max.columns = [col_ + "_max" for col_ in buro_full_max.columns]
buro_full_min.columns = [col_ + "_min" for col_ in buro_full_min.columns]
buro_full_maxmin.columns = [col_ + "_maxmin" for col_ in buro_full_maxmin.columns]

buro_full_wide = pd.concat([buro_full_avg, buro_full_max, buro_full_min, buro_full_maxmin], axis=1)
print(buro_full_wide.shape)
save_dataframe32("./bindata/buro_full_wide_013", buro_full_wide.reset_index())

del buro_full_avg, buro_full_max, buro_full_min, buro_full_maxmin, buro_full, grp
gc.collect()

(305811, 296)


214

### pos_bal_wide_013

In [16]:
prev = pd.read_csv('./data/previous_application.csv')
pos = pd.read_csv('./data/POS_CASH_balance.csv')
pos["contract_type"] = pos.SK_ID_PREV.map(prev.set_index("SK_ID_PREV").NAME_CONTRACT_TYPE)
data_stack = {}

# total group
grp = pos.groupby("SK_ID_CURR")
data_stack["apl_cnt"] = grp.SK_ID_PREV.count()
data_stack["type_cnt"] = grp.contract_type.count()
data_stack["max_month"] = grp.MONTHS_BALANCE.max()
data_stack["min_month"] = grp.MONTHS_BALANCE.min()
data_stack["maxmin_month"] = data_stack["max_month"] - data_stack["min_month"]

for active_ in pos.NAME_CONTRACT_STATUS.unique():
    grp = pos[pos.NAME_CONTRACT_STATUS == active_].groupby("SK_ID_CURR")
    data_stack["status_{}_apl_cnt".format(active_)] = grp.SK_ID_PREV.count()
    data_stack["status_{}_type_cnt".format(active_)] = grp.contract_type.count()
    data_stack["status_{}_max_month".format(active_)] = grp.MONTHS_BALANCE.max()
    data_stack["status_{}_min_month".format(active_)] = grp.MONTHS_BALANCE.min()
    data_stack["status_{}_maxmin_month".format(active_)] = data_stack["status_{}_max_month".format(active_)] - data_stack["status_{}_min_month".format(active_)]
    data_stack["status_{}_max_offset".format(active_)] = data_stack["max_month"] - data_stack["status_{}_max_month".format(active_)]
    data_stack["status_{}_min_offset".format(active_)] = data_stack["min_month"] - data_stack["status_{}_maxmin_month".format(active_)]
    data_stack["status_{}_raito".format(active_)] = data_stack["status_{}_apl_cnt".format(active_)] / data_stack["apl_cnt"]

# status group
for status_ in pos.contract_type.unique():
    grp = pos[pos.contract_type == status_].groupby("SK_ID_CURR")
    data_stack["type_{}_apl_cnt".format(status_)] = grp.SK_ID_PREV.count()
    data_stack["type_{}_max_month".format(status_)] = grp.MONTHS_BALANCE.max()
    data_stack["type_{}_min_month".format(status_)] = grp.MONTHS_BALANCE.min()
    data_stack["type_{}_maxmin_month".format(status_)] = data_stack["type_{}_max_month".format(status_)] - data_stack["type_{}_min_month".format(status_)]
    data_stack["type_{}_max_offset".format(status_)] = data_stack["max_month"] - data_stack["type_{}_max_month".format(status_)]
    data_stack["type_{}_min_offset".format(status_)] = data_stack["min_month"] - data_stack["type_{}_maxmin_month".format(status_)]
    data_stack["type_{}_raito".format(status_)] = data_stack["type_{}_apl_cnt".format(status_)] / data_stack["apl_cnt"]

pos_bal_wide = pd.DataFrame(data_stack)
pos_bal_wide.columns = ["pos_bal_" + col_ for col_ in pos_bal_wide.columns]
print(pos_bal_wide.shape)
save_dataframe32("./bindata/pos_bal_wide_013", pos_bal_wide.reset_index())

del prev, pos, grp, data_stack
gc.collect()

(337252, 98)


152

### cc_bal_wide_013

In [17]:
cc_bal = pd.read_csv('./data/credit_card_balance.csv')
# cc_bal["contract_type"] = cc_bal.SK_ID_PREV.map(prev.set_index("SK_ID_PREV").NAME_CONTRACT_TYPE)

data_stack = {}

# total group
grp = cc_bal.groupby("SK_ID_CURR")
data_stack["apl_cnt"] = grp.SK_ID_PREV.count()
data_stack["statue_cnt"] = grp.NAME_CONTRACT_STATUS.count()
data_stack["max_month"] = grp.MONTHS_BALANCE.max()
data_stack["min_month"] = grp.MONTHS_BALANCE.min()
data_stack["maxmin_month"] = data_stack["max_month"] - data_stack["min_month"]

for active_ in cc_bal.NAME_CONTRACT_STATUS.unique():
    grp = cc_bal[cc_bal.NAME_CONTRACT_STATUS == active_].groupby("SK_ID_CURR")
    data_stack["status_{}_apl_cnt".format(active_)] = grp.SK_ID_PREV.count()
    data_stack["status_{}_max_month".format(active_)] = grp.MONTHS_BALANCE.max()
    data_stack["status_{}_min_month".format(active_)] = grp.MONTHS_BALANCE.min()
    data_stack["status_{}_maxmin_month".format(active_)] = data_stack["status_{}_max_month".format(active_)] - data_stack["status_{}_min_month".format(active_)]
    data_stack["status_{}_max_offset".format(active_)] = data_stack["max_month"] - data_stack["status_{}_max_month".format(active_)]
    data_stack["status_{}_min_offset".format(active_)] = data_stack["min_month"] - data_stack["status_{}_maxmin_month".format(active_)]
    data_stack["status_{}_raito".format(active_)] = data_stack["status_{}_apl_cnt".format(active_)] / data_stack["apl_cnt"]
    
cc_bal_wide = pd.DataFrame(data_stack)
cc_bal_wide.columns = ["cc_bal_" + col_ for col_ in cc_bal_wide.columns]
print(cc_bal_wide.shape)
save_dataframe32("./bindata/cc_bal_wide_013", cc_bal_wide.reset_index())

del cc_bal, data_stack, grp
gc.collect()

(103558, 54)


245

### buro_bal_timestep1_013

In [18]:
buro_bal = pd.read_csv('./data/bureau_balance.csv')
buro = pd.read_csv('./data/bureau.csv')

buro_bal["SK_ID_CURR"] = buro_bal.SK_ID_BUREAU.map(buro.set_index("SK_ID_BUREAU").SK_ID_CURR)
buro_bal["CREDIT_TYPE"] = buro_bal.SK_ID_BUREAU.map(buro.set_index("SK_ID_BUREAU").CREDIT_TYPE)

freq_types = {
    'Consumer credit',
    'Credit card',
    'Mortgage',
    'Car loan',
    'Microloan',
    'Loan for working capital replenishment',
    'Loan for business development'
}
buro_bal["CREDIT_TYPE2"] = buro_bal.CREDIT_TYPE
buro_bal.loc[~buro_bal.CREDIT_TYPE.isin(freq_types), "CREDIT_TYPE2"] = "Others"

data_stack = {}

map_prev_step = dict(zip((2, 5, 11, 23), (0, 2, 5, 11)))

for min_month in (0, 2, 5, 11, 23):
    grp = buro_bal[buro_bal.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp.size()
    data_stack["before_{}_type_cnt".format(min_month)] = grp.CREDIT_TYPE.count()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in buro_bal.CREDIT_TYPE2.unique():
        grp = buro_bal[(buro_bal.MONTHS_BALANCE >= -min_month) & (buro_bal.CREDIT_TYPE2 == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] = grp.size()
        data_stack["before_{}_type_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_type_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_type_{}_cnt".format(min_month, type2_)]
            
buro_bal_timestep1 = pd.DataFrame(data_stack)
print(buro_bal_timestep1.shape, buro_bal_timestep1.index.nunique())
save_dataframe32("./bindata/buro_bal_timestep1_013", buro_bal_timestep1.reset_index())

del buro_bal_timestep1, data_stack, grp
gc.collect()

(133231, 126) 133231


853

### buro_bal_timestep2_013

In [19]:
buro_status = pd.get_dummies(buro_bal.STATUS).rename(columns=lambda x: "status_" + x)
buro_bal_append = pd.concat([buro_bal, buro_status], axis=1)
status_columns = [col_ for col_ in buro_bal_append.columns if col_.startswith("status")]

data_stack = {}

map_prev_step = dict(zip((2, 5, 11, 23), (0, 2, 5, 11)))

for min_month in (0, 2, 5, 11, 23):
    grp = buro_bal_append[buro_bal_append.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp[status_columns].sum()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in buro_bal.CREDIT_TYPE2.unique():
        grp = buro_bal_append[(buro_bal_append.MONTHS_BALANCE >= -min_month) & (buro_bal_append.CREDIT_TYPE2 == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] = grp[status_columns].sum()
        data_stack["before_{}_type_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_type_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_type_{}_cnt".format(min_month, type2_)]

for name_, data_ in data_stack.items():
    print(name_, data_.shape)

buro_bal_timestep2 = pd.concat([data_.rename(columns=lambda x: name_ + "_" + x) for name_, data_ in data_stack.items()], axis=1)
print(buro_bal_timestep2.shape)
save_dataframe32("./bindata/buro_bal_timestep2_013", buro_bal_timestep2.reset_index())

del buro_bal_timestep2, data_stack, grp
gc.collect()

before_0_cnt (129829, 8)
before_0_type_Credit card_cnt (78479, 8)
before_0_type_Credit card_ratio (129829, 8)
before_0_type_Consumer credit_cnt (118793, 8)
before_0_type_Consumer credit_ratio (129829, 8)
before_0_type_Car loan_cnt (7954, 8)
before_0_type_Car loan_ratio (129829, 8)
before_0_type_Others_cnt (445, 8)
before_0_type_Others_ratio (129829, 8)
before_0_type_Mortgage_cnt (6272, 8)
before_0_type_Mortgage_ratio (129829, 8)
before_0_type_Microloan_cnt (2578, 8)
before_0_type_Microloan_ratio (129829, 8)
before_0_type_Loan for working capital replenishment_cnt (242, 8)
before_0_type_Loan for working capital replenishment_ratio (129829, 8)
before_0_type_Loan for business development_cnt (439, 8)
before_0_type_Loan for business development_ratio (129829, 8)
before_2_cnt (130822, 8)
before_0_div_2_ratio (130822, 8)
before_2_type_Credit card_cnt (80018, 8)
before_2_type_Credit card_ratio (130822, 8)
before_0_div_2_type_Credit card_ratio (80018, 8)
before_2_type_Consumer credit_cnt (1202

1152

### pos_bal_timestep1_013

In [20]:
prev = pd.read_csv('./data/previous_application.csv')
pos = pd.read_csv('./data/POS_CASH_balance.csv')
pos["contract_type"] = pos.SK_ID_PREV.map(prev.set_index("SK_ID_PREV").NAME_CONTRACT_TYPE)

data_stack = {}

map_prev_step = dict(zip((2, 5, 11, 23), (0, 2, 5, 11)))

for min_month in (0, 2, 5, 11, 23):
    grp = pos[pos.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp.size()
    data_stack["before_{}_type_cnt".format(min_month)] = grp.contract_type.count()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in pos.contract_type.unique():
        grp = pos[(pos.MONTHS_BALANCE >= -min_month) & (pos.contract_type == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] = grp.size()
        data_stack["before_{}_type_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_type_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_type_{}_cnt".format(min_month, type2_)]
            
pos_bal_timestep1 = pd.DataFrame(data_stack)
print(pos_bal_timestep1.shape, pos_bal_timestep1.index.nunique())
save_dataframe32("./bindata/pos_bal_timestep1_013", pos_bal_timestep1.reset_index())

del pos_bal_timestep1, data_stack, grp
gc.collect()

(284055, 56) 284055


327

### pos_bal_timestep2_013

In [21]:
pos_numcols = list(pos.dtypes[pos.dtypes != "object"].index)
pos_numcols.remove("SK_ID_CURR")
pos_numcols.remove("SK_ID_PREV")

data_stack = {}

map_prev_step = dict(zip((5, 11, 23), (2, 5, 11)))

for min_month in (2, 5, 11, 23):
    grp = pos[pos.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp[pos_numcols].sum()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in pos.contract_type.unique():
        if str(type2_) == "nan":
            continue
        grp = pos[(pos.MONTHS_BALANCE >= -min_month) & (pos.contract_type == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] = grp[pos_numcols].sum()
        data_stack["before_{}_type_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_type_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_type_{}_cnt".format(min_month, type2_)]

for name_, data_ in data_stack.items():
    print(name_, data_.shape)

pos_timestep2 = pd.concat([data_.rename(columns=lambda x: name_ + "_" + x) for name_, data_ in data_stack.items()], axis=1)
print(pos_timestep2.shape)
save_dataframe32("./bindata/pos_bal_timestep2_013", pos_timestep2.reset_index())

del pos_timestep2, data_stack, grp
gc.collect()

before_2_cnt (146892, 5)
before_2_type_Cash loans_cnt (75264, 5)
before_2_type_Cash loans_ratio (146892, 5)
before_2_type_Consumer loans_cnt (79985, 5)
before_2_type_Consumer loans_ratio (146892, 5)
before_5_cnt (182471, 5)
before_2_div_5_ratio (182471, 5)
before_5_type_Cash loans_cnt (90777, 5)
before_5_type_Cash loans_ratio (182471, 5)
before_2_div_5_type_Cash loans_ratio (90777, 5)
before_5_type_Consumer loans_cnt (105232, 5)
before_5_type_Consumer loans_ratio (182471, 5)
before_2_div_5_type_Consumer loans_ratio (105232, 5)
before_11_cnt (233396, 5)
before_5_div_11_ratio (233396, 5)
before_11_type_Cash loans_cnt (115006, 5)
before_11_type_Cash loans_ratio (233396, 5)
before_5_div_11_type_Cash loans_ratio (115006, 5)
before_11_type_Consumer loans_cnt (146015, 5)
before_11_type_Consumer loans_ratio (233396, 5)
before_5_div_11_type_Consumer loans_ratio (146015, 5)
before_23_cnt (284055, 5)
before_11_div_23_ratio (284055, 5)
before_23_type_Cash loans_cnt (138004, 5)
before_23_type_Cash 

383

### cc_bal_timestep1_013

In [22]:
cc_bal = pd.read_csv('./data/credit_card_balance.csv')

data_stack = {}

map_prev_step = dict(zip((2, 5, 11, 23), (0, 2, 5, 11)))

for min_month in (0, 2, 5, 11, 23):
    grp = cc_bal[cc_bal.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp.size()
    data_stack["before_{}_type_cnt".format(min_month)] = grp.NAME_CONTRACT_STATUS.count()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in cc_bal.NAME_CONTRACT_STATUS.unique():
        grp = cc_bal[(cc_bal.MONTHS_BALANCE >= -min_month) & (cc_bal.NAME_CONTRACT_STATUS == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] = grp.size()
        data_stack["before_{}_type_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_type_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_type_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_type_{}_cnt".format(min_month, type2_)]
            
cc_bal_timestep1 = pd.DataFrame(data_stack)
print(cc_bal_timestep1.shape, cc_bal_timestep1.index.nunique())
save_dataframe32("./bindata/cc_bal_timestep1_013", cc_bal_timestep1.reset_index())

del cc_bal_timestep1, data_stack, grp
gc.collect()

(103558, 112) 103558


600

### cc_bal_timestep2_013

In [24]:
cc_bal["contract_status"] = cc_bal.NAME_CONTRACT_STATUS
cc_bal.loc[~cc_bal.contract_status.isin({"Active", "Completed", "Signed"}), "contract_status"] = "Other"

cc_bal_numcols = list(cc_bal.dtypes[cc_bal.dtypes != "object"].index)
cc_bal_numcols.remove("MONTHS_BALANCE")
cc_bal_numcols.remove("SK_ID_CURR")
cc_bal_numcols.remove("SK_ID_PREV")

data_stack = {}

map_prev_step = dict(zip((5, 11, 23), (2, 5, 11)))

for min_month in (2, 5, 11, 23):
    grp = cc_bal[cc_bal.MONTHS_BALANCE >= -min_month].groupby("SK_ID_CURR")
    data_stack["before_{}_cnt".format(min_month)] = grp[cc_bal_numcols].sum()
    if min_month in map_prev_step:
        prev_month = map_prev_step[min_month]
        data_stack["before_{}_div_{}_ratio".format(prev_month, min_month)] = data_stack["before_{}_cnt".format(prev_month)] / data_stack["before_{}_cnt".format(min_month)]

    for type2_ in cc_bal.contract_status.unique():
        if str(type2_) == "nan":
            continue
        grp = cc_bal[(cc_bal.MONTHS_BALANCE >= -min_month) & (cc_bal.contract_status == type2_)].groupby("SK_ID_CURR")
        data_stack["before_{}_status_{}_cnt".format(min_month, type2_)] = grp[cc_bal_numcols].sum()
        data_stack["before_{}_status_{}_ratio".format(min_month, type2_)] = data_stack["before_{}_status_{}_cnt".format(min_month, type2_)] / data_stack["before_{}_cnt".format(min_month)]
        if min_month in map_prev_step:
            prev_month = map_prev_step[min_month]
            data_stack["before_{}_div_{}_status_{}_ratio".format(prev_month, min_month, type2_)] = data_stack["before_{}_status_{}_cnt".format(prev_month, type2_)] / data_stack["before_{}_status_{}_cnt".format(min_month, type2_)]

for name_, data_ in data_stack.items():
    print(name_, data_.shape)

cc_bal_timestep2 = pd.concat([data_.rename(columns=lambda x: name_ + "_" + x) for name_, data_ in data_stack.items()], axis=1)
print(cc_bal_timestep2.shape)
save_dataframe32("./bindata/cc_bal_timestep2_013", cc_bal_timestep2.reset_index())

del cc_bal_timestep2, data_stack, grp
gc.collect()

before_2_cnt (94385, 19)
before_2_status_Active_cnt (84857, 19)
before_2_status_Active_ratio (94385, 19)
before_2_status_Completed_cnt (10530, 19)
before_2_status_Completed_ratio (94385, 19)
before_2_status_Other_cnt (17, 19)
before_2_status_Other_ratio (94385, 19)
before_2_status_Signed_cnt (46, 19)
before_2_status_Signed_ratio (94385, 19)
before_5_cnt (103556, 19)
before_2_div_5_ratio (103556, 19)
before_5_status_Active_cnt (95193, 19)
before_5_status_Active_ratio (103556, 19)
before_2_div_5_status_Active_ratio (95193, 19)
before_5_status_Completed_cnt (13179, 19)
before_5_status_Completed_ratio (103556, 19)
before_2_div_5_status_Completed_ratio (13179, 19)
before_5_status_Other_cnt (22, 19)
before_5_status_Other_ratio (103556, 19)
before_2_div_5_status_Other_ratio (22, 19)
before_5_status_Signed_cnt (187, 19)
before_5_status_Signed_ratio (103556, 19)
before_2_div_5_status_Signed_ratio (187, 19)
before_11_cnt (103557, 19)
before_5_div_11_ratio (103557, 19)
before_11_status_Active_cnt

1056

## nmf features 013

In [29]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

In [31]:
for source_name in (
    "buro_wide_013",
    "buro_full_wide_013",
    "pos_bal_wide_013",
    "cc_bal_wide_013",
    "buro_bal_timestep1_013",
    "buro_bal_timestep2_013",
    "pos_bal_timestep1_013",
    "pos_bal_timestep2_013",
    "cc_bal_timestep1_013",
    "cc_bal_timestep2_013"
):
    print(source_name)
    source_name = "./bindata/" + source_name
    target_name = source_name + "_nmf5"
    souce_data = load_dataframe32(source_name)
    columns = list(souce_data.columns)
    columns.remove("SK_ID_CURR")
    X = souce_data[columns].values
    X[np.isnan(X)] = 0
    X = MinMaxScaler().fit_transform(X)
    model = NMF(n_components=50, init='random', random_state=0)
    X_nmf5 = model.fit_transform(X)
    target_data = pd.DataFrame(X_nmf5, columns=["nmf{}".format(i) for i in range(50)])
    target_data["SK_ID_CURR"] = souce_data.SK_ID_CURR
    save_dataframe32(target_name, target_data)
    print(target_data.head())
    
    del souce_data, target_data
    gc.collect()

buro_wide_013
       nmf0      nmf1      nmf2      nmf3      nmf4      nmf5      nmf6  \
0  0.220581  0.305496  0.505983  0.000000  0.000000  0.649386  0.000054   
1  0.128088  0.320471  0.125159  0.139062  0.006342  0.383147  0.043419   
2  0.266054  0.303614  0.010646  0.072806  0.007133  0.100957  0.007241   
3  0.000000  0.318018  0.554501  0.000000  0.000000  0.000000  0.000000   
4  0.586593  0.106899  0.224221  0.054819  0.052639  0.680011  0.003229   

       nmf7  nmf8  nmf9     ...         nmf41     nmf42  nmf43     nmf44  \
0  0.000000   0.0   0.0     ...      0.005920  0.000000    0.0  0.000017   
1  0.627760   0.0   0.0     ...      0.168911  0.170890    0.0  0.000011   
2  0.745296   0.0   0.0     ...      0.113830  0.062388    0.0  0.000000   
3  0.000000   0.0   0.0     ...      0.004825  0.000288    0.0  0.000021   
4  0.211868   0.0   0.0     ...      0.000000  0.000000    0.0  0.000000   

      nmf45  nmf46  nmf47     nmf48  nmf49  SK_ID_CURR  
0  0.000014    0.0   

       nmf0      nmf1      nmf2      nmf3      nmf4      nmf5      nmf6  \
0  1.112248  0.000000  0.498110  0.000000  0.167066  0.337065  0.154409   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.585168  0.000000  0.477046  0.013394  0.157804  0.163662  0.100358   
4  0.000000  1.116239  0.000000  1.406091  0.000000  0.000000  0.000000   

           nmf7      nmf8      nmf9     ...         nmf41     nmf42     nmf43  \
0  0.000000e+00  0.000000  0.000008     ...      0.323210  0.000054  0.003824   
1  2.473818e-08  0.000000  0.195522     ...      0.000035  0.000000  0.022498   
2  0.000000e+00  0.000000  0.000000     ...      0.000107  0.000000  0.067261   
3  6.673104e-02  0.000000  0.000000     ...      0.468906  0.000000  0.015691   
4  4.083781e-02  0.606278  0.285699     ...      0.000000  0.000000  0.000000   

      nmf44     nmf45     nmf46     nmf47     nmf48     nmf49 

## single xgb

### train test split

In [32]:
data = load_dataframe32("./bindata/data_013")
data["SK_ID_CURR"] = data.SK_ID_CURR.astype("int")

test = load_dataframe32("./bindata/test_013")
test["SK_ID_CURR"] = test.SK_ID_CURR.astype("int")

y = load_dataframe32("./bindata/y_013")
y["SK_ID_CURR"] = y.SK_ID_CURR.astype("int")

In [33]:
data_train = data[data.SK_ID_CURR < 130000]
data_val = data[(data.SK_ID_CURR >= 130000) & (data.SK_ID_CURR < 150000)]

ydata_train = y[y.SK_ID_CURR < 130000]
ydata_val = y[(y.SK_ID_CURR >= 130000) & (y.SK_ID_CURR < 150000)]

In [34]:
import xgboost as xgb

class XgbWrapper:
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

### nmf features

In [35]:
for source_name in (
    "buro_wide_013",
    "buro_full_wide_013",
    "pos_bal_wide_013",
    "cc_bal_wide_013",
    "buro_bal_timestep1_013",
    "buro_bal_timestep2_013",
    "pos_bal_timestep1_013",
    "pos_bal_timestep2_013",
    "cc_bal_timestep1_013",
    "cc_bal_timestep2_013"
):
    source_name = "./bindata/" + source_name
    target_name = source_name + "_nmf5"
    nmf_data = load_dataframe32(target_name)
    nmf_data["SK_ID_CURR"] = nmf_data.SK_ID_CURR.astype("int")
    data_train = data_train.merge(right=nmf_data, how="left", on="SK_ID_CURR")
    data_val = data_val.merge(right=nmf_data, how="left", on="SK_ID_CURR")

In [36]:
data_train.shape, data_val.shape

((25803, 896), (17390, 896))

In [37]:
features = list(data_train.columns)
features.remove("SK_ID_CURR")
print(len(features))

895


In [38]:
X_num_train = data_train[features].values
X_num_val = data_val[features].values

y_train = ydata_train.TARGET.values
y_val = ydata_val.TARGET.values

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.025,
     "max_depth": 6,
     "min_child_weight": 19,
     "gamma": 0,
     "subsample": 0.8,
     "colsample_bytree": 0.632,
     "alpha": 0,
     "lambda": 0.05,
     "nrounds": 250
}

xgb_clf = XgbWrapper(params=xgb_params)
xgb_clf.train(X_num_train, y_train)
pred_val = xgb_clf.predict(X_num_val)

In [39]:
roc_auc_score(y_val, pred_val)

0.76390659652258486

### nmf + first 4

In [40]:
buro_wide = load_dataframe32("./bindata/buro_wide_013")
buro_full_wide = load_dataframe32("./bindata/buro_full_wide_013")
pos_bal_wide = load_dataframe32("./bindata/pos_bal_wide_013")
cc_bal_wide = load_dataframe32("./bindata/cc_bal_wide_013")

buro_wide["SK_ID_CURR"] = buro_wide.SK_ID_CURR.astype("int")
buro_full_wide["SK_ID_CURR"] = buro_full_wide.SK_ID_CURR.astype("int")
pos_bal_wide["SK_ID_CURR"] = pos_bal_wide.SK_ID_CURR.astype("int")
cc_bal_wide["SK_ID_CURR"] = cc_bal_wide.SK_ID_CURR.astype("int")

for data_ in (buro_wide, buro_full_wide, pos_bal_wide, cc_bal_wide):
    data_train = data_train.merge(right=data_, how="left", on="SK_ID_CURR")
    data_val = data_val.merge(right=data_, how="left", on="SK_ID_CURR")

In [41]:
data_train.shape, data_val.shape

((25803, 1423), (17390, 1423))

In [42]:
features = list(data_train.columns)
features.remove("SK_ID_CURR")
print(len(features))

1422


In [43]:
X_num_train = data_train[features].values
X_num_val = data_val[features].values

y_train = ydata_train.TARGET.values
y_val = ydata_val.TARGET.values

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.025,
     "max_depth": 6,
     "min_child_weight": 19,
     "gamma": 0,
     "subsample": 0.8,
     "colsample_bytree": 0.632,
     "alpha": 0,
     "lambda": 0.05,
     "nrounds": 250
}

xgb_clf = XgbWrapper(params=xgb_params)
xgb_clf.train(X_num_train, y_train)
pred_val = xgb_clf.predict(X_num_val)

In [44]:
roc_auc_score(y_val, pred_val)

0.76450360090667746

### append features

In [45]:
buro_bal_timestep1 = load_dataframe32("./bindata/buro_bal_timestep1_013")
pos_bal_timestep1 = load_dataframe32("./bindata/pos_bal_timestep1_013")
cc_bal_timestep1 = load_dataframe32("./bindata/cc_bal_timestep1_013")

buro_bal_timestep1["SK_ID_CURR"] = buro_bal_timestep1.SK_ID_CURR.astype("int")
pos_bal_timestep1["SK_ID_CURR"] = pos_bal_timestep1.SK_ID_CURR.astype("int")
cc_bal_timestep1["SK_ID_CURR"] = cc_bal_timestep1.SK_ID_CURR.astype("int")

In [46]:
data_train = data_train.merge(right=buro_bal_timestep1, how="left", on="SK_ID_CURR")
data_train = data_train.merge(right=pos_bal_timestep1, how="left", on="SK_ID_CURR")
data_train = data_train.merge(right=cc_bal_timestep1, how="left", on="SK_ID_CURR")

data_val = data_val.merge(right=buro_bal_timestep1, how="left", on="SK_ID_CURR")
data_val = data_val.merge(right=pos_bal_timestep1, how="left", on="SK_ID_CURR")
data_val = data_val.merge(right=cc_bal_timestep1, how="left", on="SK_ID_CURR")

In [47]:
data_train.shape, data_val.shape

((25803, 1717), (17390, 1717))

In [48]:
features = list(data_train.columns)
features.remove("SK_ID_CURR")

In [49]:
X_num_train = data_train[features].values
X_num_val = data_val[features].values

y_train = ydata_train.TARGET.values
y_val = ydata_val.TARGET.values

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.025,
     "max_depth": 6,
     "min_child_weight": 19,
     "gamma": 0,
     "subsample": 0.8,
     "colsample_bytree": 0.632,
     "alpha": 0,
     "lambda": 0.05,
     "nrounds": 250
}

xgb_clf = XgbWrapper(params=xgb_params)
xgb_clf.train(X_num_train, y_train)
pred_val = xgb_clf.predict(X_num_val)

In [50]:
roc_auc_score(y_val, pred_val)

0.76360649911384615

In [51]:
buro_bal_timestep2 = load_dataframe32("./bindata/buro_bal_timestep2_013")
pos_bal_timestep2 = load_dataframe32("./bindata/pos_bal_timestep2_013")
cc_bal_timestep2 = load_dataframe32("./bindata/cc_bal_timestep2_013")

buro_bal_timestep2["SK_ID_CURR"] = buro_bal_timestep2.SK_ID_CURR.astype("int")
pos_bal_timestep2["SK_ID_CURR"] = pos_bal_timestep2.SK_ID_CURR.astype("int")
cc_bal_timestep2["SK_ID_CURR"] = cc_bal_timestep2.SK_ID_CURR.astype("int")

In [52]:
data_train = data_train.merge(right=buro_bal_timestep2, how="left", on="SK_ID_CURR")
data_train = data_train.merge(right=pos_bal_timestep2, how="left", on="SK_ID_CURR")
data_train = data_train.merge(right=cc_bal_timestep2, how="left", on="SK_ID_CURR")

data_val = data_val.merge(right=buro_bal_timestep2, how="left", on="SK_ID_CURR")
data_val = data_val.merge(right=pos_bal_timestep2, how="left", on="SK_ID_CURR")
data_val = data_val.merge(right=cc_bal_timestep2, how="left", on="SK_ID_CURR")

In [53]:
data_train.shape, data_val.shape

((25803, 3799), (17390, 3799))

In [54]:
features = list(data_train.columns)
features.remove("SK_ID_CURR")

In [55]:
X_num_train = data_train[features].values
X_num_val = data_val[features].values

y_train = ydata_train.TARGET.values
y_val = ydata_val.TARGET.values

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.025,
     "max_depth": 6,
     "min_child_weight": 19,
     "gamma": 0,
     "subsample": 0.8,
     "colsample_bytree": 0.632,
     "alpha": 0,
     "lambda": 0.05,
     "nrounds": 250
}

xgb_clf = XgbWrapper(params=xgb_params)
xgb_clf.train(X_num_train, y_train)
pred_val = xgb_clf.predict(X_num_val)

In [56]:
roc_auc_score(y_val, pred_val)

0.76365094665840128

In [57]:
buro_wide = load_dataframe32("./bindata/buro_wide_013")
buro_full_wide = load_dataframe32("./bindata/buro_full_wide_013")
pos_bal_wide = load_dataframe32("./bindata/pos_bal_wide_013")
cc_bal_wide = load_dataframe32("./bindata/cc_bal_wide_013")

buro_wide["SK_ID_CURR"] = buro_wide.SK_ID_CURR.astype("int")
buro_full_wide["SK_ID_CURR"] = buro_full_wide.SK_ID_CURR.astype("int")
pos_bal_wide["SK_ID_CURR"] = pos_bal_wide.SK_ID_CURR.astype("int")
cc_bal_wide["SK_ID_CURR"] = cc_bal_wide.SK_ID_CURR.astype("int")

for data_ in (buro_wide, buro_full_wide, pos_bal_wide, cc_bal_wide):
    data_train = data_train.merge(right=data_, how="left", on="SK_ID_CURR")
    data_val = data_val.merge(right=data_, how="left", on="SK_ID_CURR")

In [58]:
features = list(data_train.columns)
features.remove("SK_ID_CURR")

In [59]:
data_train.shape, data_val.shape

((25803, 4326), (17390, 4326))

In [60]:
X_num_train = data_train[features].values
X_num_val = data_val[features].values

y_train = ydata_train.TARGET.values
y_val = ydata_val.TARGET.values

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.025,
     "max_depth": 6,
     "min_child_weight": 19,
     "gamma": 0,
     "subsample": 0.8,
     "colsample_bytree": 0.632,
     "alpha": 0,
     "lambda": 0.05,
     "nrounds": 250
}

xgb_params = {
     "objective": "binary:logistic",
     "booster": "gbtree",
     "eval_metric": "auc",
     "nthread": 8,
     "eta": 0.02,
     "max_depth": 8,
     "min_child_weight": 40,
     "gamma": 0,
     "subsample": 0.8715623,
     "colsample_bytree": 0.632,
     "alpha": 0.041545473,
     "lambda": 0.0735294,
     "nrounds": 250
}

xgb_clf = XgbWrapper(params=xgb_params)
xgb_clf.train(X_num_train, y_train)
pred_val = xgb_clf.predict(X_num_val)

In [61]:
roc_auc_score(y_val, pred_val)

0.76432342882956028

## stacking

https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm